# Feature engineering

## Setup

In [5]:
%pip install -r ../requirements.txt

In [6]:
import sys
sys.path.append('../')

In [7]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm # DO NOT REMOVE

%matplotlib inline

The autoreload module is not an IPython extension.


## Run setup


In [17]:
CSV_PATH = "../data/gym_members_exercise_tracking.csv"
df = pd.read_csv(CSV_PATH)

In [19]:
from libs.setup import Run

run = Run(Path.joinpath(Path.cwd().parent, "runs")).initialize()

run.add_raw(df, filename="gym_members_exercise_tracking")

WindowsPath('c:/Users/drwni/Desktop/MEINF/AI/WP_01/runs/2024-12-12 20-12-34/data/raw/gym_members_exercise_tracking')

## Feature creation

In [10]:
# Show missing values
_null = df.isnull().sum()

print(_null[_null > 0] if any(_null) else "No missing values")

No missing values


There are no missing values in the dataset, so we won't have to create any missing imputation.

## Feature encoding

In [20]:
from sklearn.preprocessing import OneHotEncoder

_encoder = OneHotEncoder()
_encoded_categorical = pd.DataFrame(
    _encoder.fit_transform(df.select_dtypes(include=['object'])).toarray(), 
    columns=_encoder.get_feature_names_out(df.select_dtypes(include=['object']).columns),
)

encoded_df = df.drop(columns=df.select_dtypes(include=['object']).columns)
encoded_df = pd.concat([encoded_df, _encoded_categorical], axis=1)
encoded_df.head()

,Age,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI,Gender_Female,Gender_Male,Workout_Type_Cardio,Workout_Type_HIIT,Workout_Type_Strength,Workout_Type_Yoga
0,56,88.3,1.71,180,157,60,1.69,1313.0,12.6,3.5,4,3,30.20,0.0,1.0,0.0,0.0,0.0,1.0
1,46,74.9,1.53,179,151,66,1.30,883.0,33.9,2.1,4,2,32.00,1.0,0.0,0.0,1.0,0.0,0.0
2,32,68.1,1.66,167,122,54,1.11,677.0,33.4,2.3,4,2,24.71,1.0,0.0,1.0,0.0,0.0,0.0
3,25,53.2,1.70,190,164,56,0.59,532.0,28.8,2.1,3,1,18.41,0.0,1.0,0.0,0.0,1.0,0.0
4,38,46.1,1.79,188,158,68,0.64,556.0,29.2,2.8,3,1,14.39,0.0,1.0,0.0,0.0,1.0,0.0


## Feature selection